# Combine multiple data sources

to create a site_code & product_code & date level logistics data with time variant and time invariant exogenous features 


- The last step of the data cleaning 
- combines data sources from previous data cleaning process 
- output is used in model XXX 
- pad time_variant supplement data to include the last 3 months input 

## Import: from datacleaning_upsample step  "..\2_pipeline\df_upsample.csv" 

In [1]:
pwd

'C:\\Users\\Herbie Huang\\Dropbox (Kenan-Flagler)\\USAID_Forecast_submit\\1_code'

In [2]:
import pandas as pd
df_upsample=pd.read_csv("../2_pipeline/df_upsample.csv")

In [3]:
df_upsample=df_upsample.drop(columns=['Unnamed: 0'])
df_upsample.head()
df_upsample['date']=pd.to_datetime(df_upsample['date'])

In [4]:
df_upsample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41348 entries, 0 to 41347
Data columns (total 8 columns):
site_code            41348 non-null object
product_code         41348 non-null object
date                 41348 non-null datetime64[ns]
year                 41348 non-null int64
month                41348 non-null int64
stock_distributed    41348 non-null float64
region               41348 non-null object
district             41348 non-null object
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 2.5+ MB


## Import: data for submission format '../0_data/submission_format.csv'

In [5]:
submission_format=pd.read_csv('../0_data/submission_format.csv')

In [6]:
submission_format.head()

,year,month,site_code,product_code,predicted_value
0,2019,10,C4001,AS27134,0
1,2019,10,C4001,AS27132,0
2,2019,10,C4001,AS27000,0
3,2019,10,C4001,AS27137,0
4,2019,10,C4001,AS27138,0


In [7]:
len(submission_format[['site_code','product_code']].drop_duplicates())

1052

In [8]:
temp_site_product=submission_format[['site_code','product_code','predicted_value']].drop_duplicates()

In [9]:
temp_site_product['year']=2019
temp_month10=temp_site_product.copy()
temp_month10['month']=10
temp_month11=temp_site_product.copy()
temp_month11['month']=11
temp_month12=temp_site_product.copy()
temp_month12['month']=12
temp_site_product=pd.concat([temp_month10,temp_month11,temp_month12],ignore_index=True)

In [10]:
len(temp_site_product)

3156

In [11]:
temp_site_product.tail()

,site_code,product_code,predicted_value,year,month
3151,C5076,AS27139,0,2019,12
3152,C5076,AS27137,0,2019,12
3153,C5076,AS27138,0,2019,12
3154,C5076,AS27133,0,2019,12
3155,C5076,AS46000,0,2019,12


In [12]:
submission_format=temp_site_product.copy()
submission_format['day']=1
submission_format['date']=pd.to_datetime(submission_format[['year','month','day']])
submission_format=submission_format.sort_values('date')
submission_format.set_index('date',inplace=True )
submission_format.head(2)


,site_code,product_code,predicted_value,year,month,day
date,,,,,,
2019-10-01,C4001,AS27134,0,2019,10,1
2019-10-01,C5063,AS46000,0,2019,10,1


In [13]:
submission_format_copy=submission_format.copy()
submission_format_copy=submission_format_copy.groupby(['site_code','product_code']).resample('MS').asfreq() 
submission_format_copy=submission_format_copy.drop(['product_code','site_code'],1).reset_index().drop(columns=['day'])         
submission_format_copy.tail()

,site_code,product_code,date,predicted_value,year,month
3151,C5076,AS27139,2019-11-01,0,2019,11
3152,C5076,AS27139,2019-12-01,0,2019,12
3153,C5076,AS46000,2019-10-01,0,2019,10
3154,C5076,AS46000,2019-11-01,0,2019,11
3155,C5076,AS46000,2019-12-01,0,2019,12


- same length. So the submission data doesn't have time gaps

In [14]:
len(submission_format_copy)

3156

### add region and district to submission format 

In [15]:
temp_site_region_district=df_upsample[['site_code','region','district']].drop_duplicates().reset_index().drop(columns=['index']).copy(deep=True)

submission_format_copy=pd.merge(left=submission_format_copy,right=temp_site_region_district,on=['site_code'],how='left')

submission_format_copy.rename(columns={'predicted_value':'stock_distributed'},inplace=True)

submission_format_copy.tail()

,site_code,product_code,date,stock_distributed,year,month,region,district
3151,C5076,AS27139,2019-11-01,0,2019,11,KABADOUGOU-BAFING-FOLON,KORO
3152,C5076,AS27139,2019-12-01,0,2019,12,KABADOUGOU-BAFING-FOLON,KORO
3153,C5076,AS46000,2019-10-01,0,2019,10,KABADOUGOU-BAFING-FOLON,KORO
3154,C5076,AS46000,2019-11-01,0,2019,11,KABADOUGOU-BAFING-FOLON,KORO
3155,C5076,AS46000,2019-12-01,0,2019,12,KABADOUGOU-BAFING-FOLON,KORO


In [16]:
submission_format_copy=submission_format_copy[['site_code','product_code','date','year','month','stock_distributed','region','district']]

In [17]:
submission_format_copy.head()

,site_code,product_code,date,year,month,stock_distributed,region,district
0,C1004,AS21126,2019-10-01,2019,10,0,AGNEBY-TIASSA-ME,AGBOVILLE
1,C1004,AS21126,2019-11-01,2019,11,0,AGNEBY-TIASSA-ME,AGBOVILLE
2,C1004,AS21126,2019-12-01,2019,12,0,AGNEBY-TIASSA-ME,AGBOVILLE
3,C1004,AS27000,2019-10-01,2019,10,0,AGNEBY-TIASSA-ME,AGBOVILLE
4,C1004,AS27000,2019-11-01,2019,11,0,AGNEBY-TIASSA-ME,AGBOVILLE


In [18]:
df_upsample.head()

,site_code,product_code,date,year,month,stock_distributed,region,district
0,C1004,AS21126,2019-04-01,2019,4,0.0,AGNEBY-TIASSA-ME,AGBOVILLE
1,C1004,AS21126,2019-05-01,2019,5,0.0,AGNEBY-TIASSA-ME,AGBOVILLE
2,C1004,AS21126,2019-06-01,2019,6,0.0,AGNEBY-TIASSA-ME,AGBOVILLE
3,C1004,AS21126,2019-07-01,2019,7,0.0,AGNEBY-TIASSA-ME,AGBOVILLE
4,C1004,AS21126,2019-08-01,2019,8,0.0,AGNEBY-TIASSA-ME,AGBOVILLE


In [19]:
df_upsample_copy=pd.concat([df_upsample,submission_format_copy]).reset_index()

In [20]:
df_upsample_copy.tail()

,index,site_code,product_code,date,year,month,stock_distributed,region,district
44499,3151,C5076,AS27139,2019-11-01,2019,11,0.0,KABADOUGOU-BAFING-FOLON,KORO
44500,3152,C5076,AS27139,2019-12-01,2019,12,0.0,KABADOUGOU-BAFING-FOLON,KORO
44501,3153,C5076,AS46000,2019-10-01,2019,10,0.0,KABADOUGOU-BAFING-FOLON,KORO
44502,3154,C5076,AS46000,2019-11-01,2019,11,0.0,KABADOUGOU-BAFING-FOLON,KORO
44503,3155,C5076,AS46000,2019-12-01,2019,12,0.0,KABADOUGOU-BAFING-FOLON,KORO


In [21]:
df_upsample=df_upsample_copy.copy()

## Import - from data cleaning supplement - time invariant 

In [22]:
time_invariant_supplement=pd.read_stata("../0_data/time_invariant_supplement.dta")

In [23]:
time_invariant_supplement.head()

,region,district,site_code,product_code,product_type,regionroads,regionasphaltroads,regionearthroads,regionsurfacetreatmentroads,regionpoorroads,poorroads,earthroads,asphaltroads
0,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS21126,Injectable Contraceptive,98,24,56,18,8,1.0,0.0,1.0
1,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27000,Oral Contraceptive (Pill),98,24,56,18,8,1.0,0.0,1.0
2,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27132,Oral Contraceptive (Pill),98,24,56,18,8,1.0,0.0,1.0
3,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27133,Injectable Contraceptive,98,24,56,18,8,1.0,0.0,1.0
4,AGNEBY-TIASSA-ME,AGBOVILLE,C1004,AS27134,Injectable Contraceptive,98,24,56,18,8,1.0,0.0,1.0


## Import - site information 
### add site_type 

In [24]:
site_type=pd.read_csv('../0_data/service_delivery_site_data.csv')[['site_code','site_type']].drop_duplicates()

In [25]:
site_type.tail()

,site_code,site_type
151,C2071,Hospital
152,C1101,Hospital
153,C1106,University Hospital/National Institute
154,C1144,Health Center
155,C1679,Hospital


## Product_type  information

In [27]:
product_type=pd.read_csv('../0_data/product.csv')[['product_code','product_type']].drop_duplicates()

## Merge - df_upsample <-time_invariant_supplement (time invariant left on df_upsample) 

In [29]:
df_combined=pd.merge(left=df_upsample,right=time_invariant_supplement,on=['region','district','site_code','product_code'],how='left')
df_combined=pd.merge(left=df_combined,right=site_type,on=['site_code'],how='left')
df_combined=pd.merge(left=df_combined.drop(columns=['product_type']),right=product_type,on=['product_code'],how='left')

In [30]:
df_combined.head()

,index,site_code,product_code,date,year,month,stock_distributed,region,district,regionroads,regionasphaltroads,regionearthroads,regionsurfacetreatmentroads,regionpoorroads,poorroads,earthroads,asphaltroads,site_type,product_type
0,0,C1004,AS21126,2019-04-01,2019,4,0.0,AGNEBY-TIASSA-ME,AGBOVILLE,98.0,24.0,56.0,18.0,8.0,1.0,0.0,1.0,Hospital,Injectable Contraceptive
1,1,C1004,AS21126,2019-05-01,2019,5,0.0,AGNEBY-TIASSA-ME,AGBOVILLE,98.0,24.0,56.0,18.0,8.0,1.0,0.0,1.0,Hospital,Injectable Contraceptive
2,2,C1004,AS21126,2019-06-01,2019,6,0.0,AGNEBY-TIASSA-ME,AGBOVILLE,98.0,24.0,56.0,18.0,8.0,1.0,0.0,1.0,Hospital,Injectable Contraceptive
3,3,C1004,AS21126,2019-07-01,2019,7,0.0,AGNEBY-TIASSA-ME,AGBOVILLE,98.0,24.0,56.0,18.0,8.0,1.0,0.0,1.0,Hospital,Injectable Contraceptive
4,4,C1004,AS21126,2019-08-01,2019,8,0.0,AGNEBY-TIASSA-ME,AGBOVILLE,98.0,24.0,56.0,18.0,8.0,1.0,0.0,1.0,Hospital,Injectable Contraceptive


## Import - from data cleaning supplement - time variant 

In [31]:
time_variant_supplement=pd.read_stata("../0_data/time_variant_supplement.dta")
time_variant_supplement.tail()

,year,month,region,district,site_code,product_code,temp,maxtemp,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog,temp_timeindex
41343,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27132,25.0,29.4,1011.599976,85.699997,240.779999,8.4,3.6,9.0,13.0,20.0,3.0,716.0
41344,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27133,25.0,29.4,1011.599976,85.699997,240.779999,8.4,3.6,9.0,13.0,20.0,3.0,716.0
41345,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27137,25.0,29.4,1011.599976,85.699997,240.779999,8.4,3.6,9.0,13.0,20.0,3.0,716.0
41346,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27138,25.0,29.4,1011.599976,85.699997,240.779999,8.4,3.6,9.0,13.0,20.0,3.0,716.0
41347,2019,9,KABADOUGOU-BAFING-FOLON,KORO,C5076,AS27139,25.0,29.4,1011.599976,85.699997,240.779999,8.4,3.6,9.0,13.0,20.0,3.0,716.0


- it's obviously not long enough , only 2993 obs, not 3115 
- just ignore this caveat for now, pad time_variant this way and fillna with mean

In [32]:
time_variant_supplement_3moremonth=time_variant_supplement[(time_variant_supplement['year']==2018)&(time_variant_supplement['month']>9)].copy(deep=True)
#manually change the year to 2019
time_variant_supplement_3moremonth['year']=2019
time_variant_supplement_3moremonth.describe()

,year,month,temp,maxtemp,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog,temp_timeindex
count,3020.0,3020.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,2957.000000,3020.000000
mean,2019.0,11.010265,27.208708,31.594826,593.584473,77.836006,49.371952,8.589896,6.396868,12.638869,5.931349,9.478526,1.904633,706.010254
std,0.0,0.817716,1.005423,1.165516,497.898376,10.989476,59.884148,1.279120,4.005072,7.817459,6.359363,7.647775,4.152828,0.817723
min,2019.0,10.000000,24.200001,28.900000,0.000000,30.200001,0.000000,4.800000,0.000000,0.000000,0.000000,0.000000,0.000000,705.000000
25%,2019.0,10.000000,26.700001,30.900000,0.000000,78.699997,0.510000,7.600000,2.900000,7.500000,1.000000,2.000000,0.000000,705.000000
50%,2019.0,11.000000,27.200001,31.500000,1009.700012,81.300003,21.080000,9.200000,6.800000,12.700000,2.000000,7.000000,0.000000,706.000000
75%,2019.0,12.000000,28.200001,32.200001,1011.099976,83.099998,136.649994,9.600000,10.600000,20.900000,14.000000,19.000000,1.000000,707.000000
max,2019.0,12.000000,28.700001,35.400002,1016.299988,88.800003,229.880005,10.000000,13.300000,27.799999,22.000000,27.000000,19.000000,707.000000


In [33]:
time_variant_supplement=pd.concat([time_variant_supplement,time_variant_supplement_3moremonth]).reset_index(drop=True)

## Merge : df_combined<-time variant 

In [34]:
df_combined=pd.merge(left=df_combined,right=time_variant_supplement,on=['region','district','site_code','product_code','month','year'],how='left')

- 41348 seems to be the 'correct' obs number

In [35]:
df_combined.describe()

,index,year,month,stock_distributed,regionroads,regionasphaltroads,regionearthroads,regionsurfacetreatmentroads,regionpoorroads,poorroads,...,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog,temp_timeindex
count,44504.000000,44504.000000,44504.000000,44504.000000,44486.000000,44486.000000,44486.000000,44486.000000,44486.000000,44486.000000,...,43225.000000,43225.000000,43225.000000,43225.000000,43225.000000,43225.000000,43225.000000,43225.000000,43225.000000,44133.000000
mean,19319.307658,2017.627045,6.638841,13.038221,97.381154,12.278964,69.424538,15.677651,4.905678,0.459650,...,598.904724,78.269577,91.836731,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337,696.003052
std,12508.247874,1.095467,3.434310,38.266454,57.268333,9.990077,52.573777,10.759343,6.402652,0.498343,...,497.259247,13.451122,92.216423,1.439245,4.631456,8.408489,6.531198,7.198279,2.903406,12.458749
min,0.000000,2016.000000,1.000000,0.000000,15.000000,0.000000,9.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,672.000000
25%,7969.750000,2017.000000,4.000000,0.000000,72.000000,0.000000,34.000000,8.000000,0.000000,0.000000,...,0.000000,78.400002,26.160000,8.300000,3.200000,8.800000,3.000000,2.000000,0.000000,686.000000
50%,19095.500000,2018.000000,7.000000,0.000000,80.000000,15.000000,56.000000,15.000000,0.000000,0.000000,...,1009.799988,81.900002,63.250000,9.000000,7.700000,15.300000,8.000000,7.000000,0.000000,697.000000
75%,30221.250000,2019.000000,10.000000,10.000000,131.000000,24.000000,96.000000,19.000000,9.000000,1.000000,...,1012.000000,85.199997,136.649994,9.500000,11.700000,21.600000,14.000000,14.000000,1.000000,707.000000
max,41347.000000,2019.000000,12.000000,1805.000000,206.000000,25.000000,184.000000,51.000000,19.000000,1.000000,...,1016.700012,129.020004,870.200012,10.000000,21.000000,39.099998,26.000000,27.000000,23.000000,716.000000


## Fillna 

In [36]:
df_combined.columns

Index(['index', 'site_code', 'product_code', 'date', 'year', 'month',
       'stock_distributed', 'region', 'district', 'regionroads',
       'regionasphaltroads', 'regionearthroads', 'regionsurfacetreatmentroads',
       'regionpoorroads', 'poorroads', 'earthroads', 'asphaltroads',
       'site_type', 'product_type', 'temp', 'maxtemp', 'pressure',
       'relativehumidity', 'rainfallsnowmelt', 'visibility', 'windspeed',
       'maxsustainedwindspeed', 'rainordrizzle', 'storm', 'fog',
       'temp_timeindex'],
      dtype='object')

In [37]:
type(df_combined['district'].iloc[0])

str

We are filling NA here because it's not too bad and we don't want to deal with them later. It seems that the NA values occured mostly for the time variant variables. An opportunity for later fix 

In [39]:
for col in ['regionroads', 'regionasphaltroads', 'regionearthroads',
       'regionsurfacetreatmentroads', 'regionpoorroads', 'poorroads',
       'earthroads', 'asphaltroads', 'temp', 'maxtemp',
       'pressure', 'relativehumidity', 'rainfallsnowmelt', 'visibility',
       'windspeed', 'maxsustainedwindspeed', 'rainordrizzle', 'storm', 'fog']:
    print(col)
    df_combined[col].fillna(value=df_combined[col].mean(),inplace=True)

regionroads
regionasphaltroads
regionearthroads
regionsurfacetreatmentroads
regionpoorroads
poorroads
earthroads
asphaltroads
temp
maxtemp
pressure
relativehumidity
rainfallsnowmelt
visibility
windspeed
maxsustainedwindspeed
rainordrizzle
storm
fog


In [40]:
df_combined.describe()

,index,year,month,stock_distributed,regionroads,regionasphaltroads,regionearthroads,regionsurfacetreatmentroads,regionpoorroads,poorroads,...,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog,temp_timeindex
count,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,...,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44133.000000
mean,19319.307658,2017.627045,6.638841,13.038221,97.381154,12.278964,69.424538,15.677651,4.905678,0.459650,...,598.900146,78.269035,91.835045,8.567291,7.227873,14.248749,8.856819,8.747553,1.313337,696.003052
std,12508.247874,1.095467,3.434310,38.266454,57.256750,9.988057,52.563144,10.757167,6.401357,0.498242,...,490.061584,13.256230,90.882072,1.418491,4.564510,8.286765,6.436662,7.094087,2.861380,12.458749
min,0.000000,2016.000000,1.000000,0.000000,15.000000,0.000000,9.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,672.000000
25%,7969.750000,2017.000000,4.000000,0.000000,72.000000,0.000000,34.000000,8.000000,0.000000,0.000000,...,0.000000,78.269577,26.670000,8.300000,3.475000,9.200000,3.000000,2.000000,0.000000,686.000000
50%,19095.500000,2018.000000,7.000000,0.000000,80.000000,15.000000,56.000000,15.000000,0.000000,0.000000,...,1009.599976,81.800003,66.279999,9.000000,7.400000,15.200000,8.856819,7.000000,0.000000,697.000000
75%,30221.250000,2019.000000,10.000000,10.000000,131.000000,24.000000,96.000000,19.000000,9.000000,1.000000,...,1011.900024,85.099998,136.649994,9.500000,11.600000,21.600000,14.000000,14.000000,1.000000,707.000000
max,41347.000000,2019.000000,12.000000,1805.000000,206.000000,25.000000,184.000000,51.000000,19.000000,1.000000,...,1016.700012,129.020004,870.200012,10.000000,21.000000,39.099998,26.000000,27.000000,23.000000,716.000000


In [41]:
df_combined=df_combined.drop(columns=['temp_timeindex','index'])

In [42]:
df_combined.describe()

,year,month,stock_distributed,regionroads,regionasphaltroads,regionearthroads,regionsurfacetreatmentroads,regionpoorroads,poorroads,earthroads,...,maxtemp,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog
count,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,...,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000,44504.000000
mean,2017.627045,6.638841,13.038221,97.381154,12.278964,69.424538,15.677651,4.905678,0.459650,0.503956,...,31.508608,598.900146,78.269035,91.835045,8.567291,7.227873,14.248749,8.856819,8.747553,1.313337
std,1.095467,3.434310,38.266454,57.256750,9.988057,52.563144,10.757167,6.401357,0.498242,0.499890,...,3.048696,490.061584,13.256230,90.882072,1.418491,4.564510,8.286765,6.436662,7.094087,2.861380
min,2016.000000,1.000000,0.000000,15.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.000000,4.000000,0.000000,72.000000,0.000000,34.000000,8.000000,0.000000,0.000000,0.000000,...,29.900000,0.000000,78.269577,26.670000,8.300000,3.475000,9.200000,3.000000,2.000000,0.000000
50%,2018.000000,7.000000,0.000000,80.000000,15.000000,56.000000,15.000000,0.000000,0.000000,1.000000,...,31.500000,1009.599976,81.800003,66.279999,9.000000,7.400000,15.200000,8.856819,7.000000,0.000000
75%,2019.000000,10.000000,10.000000,131.000000,24.000000,96.000000,19.000000,9.000000,1.000000,1.000000,...,32.799999,1011.900024,85.099998,136.649994,9.500000,11.600000,21.600000,14.000000,14.000000,1.000000
max,2019.000000,12.000000,1805.000000,206.000000,25.000000,184.000000,51.000000,19.000000,1.000000,1.000000,...,64.599998,1016.700012,129.020004,870.200012,10.000000,21.000000,39.099998,26.000000,27.000000,23.000000


In [43]:
df_combined.tail()

,site_code,product_code,date,year,month,stock_distributed,region,district,regionroads,regionasphaltroads,...,maxtemp,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog
44499,C5076,AS27139,2019-11-01,2019,11,0.0,KABADOUGOU-BAFING-FOLON,KORO,187.000000,17.000000,...,31.508921,598.904724,78.269577,91.836731,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337
44500,C5076,AS27139,2019-12-01,2019,12,0.0,KABADOUGOU-BAFING-FOLON,KORO,187.000000,17.000000,...,31.508921,598.904724,78.269577,91.836731,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337
44501,C5076,AS46000,2019-10-01,2019,10,0.0,KABADOUGOU-BAFING-FOLON,KORO,97.381154,12.278964,...,31.508921,598.904724,78.269577,91.836731,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337
44502,C5076,AS46000,2019-11-01,2019,11,0.0,KABADOUGOU-BAFING-FOLON,KORO,97.381154,12.278964,...,31.508921,598.904724,78.269577,91.836731,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337
44503,C5076,AS46000,2019-12-01,2019,12,0.0,KABADOUGOU-BAFING-FOLON,KORO,97.381154,12.278964,...,31.508921,598.904724,78.269577,91.836731,8.567444,7.227927,14.248726,8.856819,8.747553,1.313337


## Export - the training data for model building (remove last 3 month empty) 


In [44]:
df_combined.set_index('date')[:'2019-9'].describe()

,year,month,stock_distributed,regionroads,regionasphaltroads,regionearthroads,regionsurfacetreatmentroads,regionpoorroads,poorroads,earthroads,...,maxtemp,pressure,relativehumidity,rainfallsnowmelt,visibility,windspeed,maxsustainedwindspeed,rainordrizzle,storm,fog
count,41348.00000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,...,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000,41348.000000
mean,2017.52225,6.305964,14.033399,97.404639,12.268139,69.451678,15.684822,4.927107,0.460071,0.505563,...,31.503555,600.287659,78.302231,94.549072,8.565675,7.278142,14.345420,9.040154,8.688441,1.278648
std,1.06620,3.328858,39.523746,57.200036,9.981447,52.494215,10.777428,6.420218,0.498457,0.499960,...,3.149080,491.869537,13.457779,92.343475,1.434456,4.617491,8.348988,6.428582,7.084147,2.774533
min,2016.00000,1.000000,0.000000,15.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.00000,3.000000,0.000000,72.000000,0.000000,34.000000,8.000000,0.000000,0.000000,0.000000,...,29.799999,0.000000,78.269577,26.930000,8.400000,3.400000,9.200000,3.000000,2.000000,0.000000
50%,2018.00000,6.000000,0.000000,80.000000,15.000000,56.000000,15.000000,0.000000,0.000000,1.000000,...,31.508921,1009.700012,81.900002,68.570000,9.000000,7.500000,15.300000,8.856819,7.000000,0.000000
75%,2018.00000,9.000000,12.000000,131.000000,24.000000,96.000000,19.000000,9.000000,1.000000,1.000000,...,32.900002,1012.099976,85.300003,136.649994,9.500000,11.700000,21.600000,14.000000,14.000000,1.000000
max,2019.00000,12.000000,1805.000000,206.000000,25.000000,184.000000,51.000000,19.000000,1.000000,1.000000,...,64.599998,1016.700012,129.020004,870.200012,10.000000,21.000000,39.099998,26.000000,27.000000,23.000000


In [45]:
df_combined.set_index('date')[:'2019-9'].to_csv('../0_data/df_training.csv')

Confirmed that the df_combined has the same # of rows as the data submission format 

## Export : full data - including the last 3 months of exog variables 

In [46]:
df_combined.set_index('date').to_csv('../0_data/df_combined_fullsample.csv')